# Reactant.jl integration

Reactant.jl is a Julia frontend to the MLIR framework and XLA compiler. It takes Julia code, optimizes aggresively and compiles to CPU / GPU / TPU / distributed clusters.

In [10]:
using TenetCore
using Reactant
using Adapt
using Random
using Chairmarks

In [19]:
Random.seed!(1234)
tn = rand(TensorNetwork, 4, 3)

GenericTensorNetwork (#tensors=4, #inds=6)

Reactant.jl traces the use of its own array types; i.e. it only tracks the operations performed on `Reactant.ConcreteRArray`s.

So first, the arrays of tensors in the Tensor Network need to be converted to `ConcreteRArray`. This is easily done with `Adapt.adapt`.

In [20]:
tnre = adapt(ConcreteRArray, tn)

GenericTensorNetwork (#tensors=4, #inds=6)

In [21]:
@info "before" typeof(tensors(tn)[1])
@info "after" typeof(tensors(tnre)[1])

┌ Info: before
│   typeof((tensors(tn))[1]) = Tensor{Float64, 6, Array{Float64, 6}}
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:1
┌ Info: after
│   typeof((tensors(tnre))[1]) = Tensor{Float64, 6, ConcretePJRTArray{Float64, 6, 1, Reactant.Sharding.ShardInfo{Reactant.Sharding.NoSharding, Nothing}}}
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:2


`Reactant.compile` compiles the given code using the compilers mentioned above and returns a _thunk_ (a _functor_, a callable object). The thunk accepts the same type of arguments as the ones passed to `Reactant.compile`.

In [22]:
f = Reactant.compile(TenetCore.contract, (tnre,))
f(tnre)

0-dimensional Tensor{Float64, 0, ConcretePJRTArray{Float64, 0, 1, Reactant.Sharding.ShardInfo{Reactant.Sharding.NoSharding, Nothing}}}:
1291.8407836130243

`@compile` is a helper macro for `Reactant.compile`.

In [23]:
f = @compile TenetCore.contract(tnre)
f(tnre)

0-dimensional Tensor{Float64, 0, ConcretePJRTArray{Float64, 0, 1, Reactant.Sharding.ShardInfo{Reactant.Sharding.NoSharding, Nothing}}}:
1291.8407836130243

`@jit` is a helper macro that is equivalent to `@compile` plus directly calling the compiled function.

In [24]:
@jit TenetCore.contract(tnre)

0-dimensional Tensor{Float64, 0, ConcretePJRTArray{Float64, 0, 1, Reactant.Sharding.ShardInfo{Reactant.Sharding.NoSharding, Nothing}}}:
1291.8407836130243

In [25]:
@b f(tnre)

7.125 μs (46 allocs: 1.562 KiB)

## Inspecting what's happenning under the hood

Just like with Julia's `@code_lowered`, `@code_llvm`, `@code_native`, ... introspection tools, Reactant.jl provides the `@code_hlo`, `@code_mhlo` and `@code_xla` tools to inspect the emitted MLIR and HLO code.

Despite its name, `@code_hlo` prints the MLIR representation of the code, which is the main representation Reactant.jl works with. For example, the MLIR code of the compiled contraction is:

In [ ]:
@code_hlo TenetCore.contract(tnre)

module @reactant_contract attributes {mhlo.num_partitions = 1 : i64, mhlo.num_replicas = 1 : i64} {
  func.func @main(%arg0: tensor<7x6x7x8x5x2xf64>, %arg1: tensor<2x7xf64>, %arg2: tensor<6x7xf64>, %arg3: tensor<5x8xf64>) -> tensor<f64> {
    %0 = stablehlo.transpose %arg0, dims = [5, 4, 3, 2, 1, 0] : (tensor<7x6x7x8x5x2xf64>) -> tensor<2x5x8x7x6x7xf64>
    %1 = stablehlo.transpose %arg1, dims = [1, 0] : (tensor<2x7xf64>) -> tensor<7x2xf64>
    %2 = stablehlo.transpose %arg2, dims = [1, 0] : (tensor<6x7xf64>) -> tensor<7x6xf64>
    %3 = stablehlo.transpose %arg3, dims = [1, 0] : (tensor<5x8xf64>) -> tensor<8x5xf64>
    %4 = stablehlo.dot_general %0, %2, contracting_dims = [3, 4] x [0, 1], precision = [DEFAULT, DEFAULT] : (tensor<2x5x8x7x6x7xf64>, tensor<7x6xf64>) -> tensor<2x5x8x7xf64>
    %5 = stablehlo.dot_general %4, %3, contracting_dims = [1, 2] x [1, 0], precision = [DEFAULT, DEFAULT] : (tensor<2x5x8x7xf64>, tensor<8x5xf64>) -> tensor<2x7xf64>
    %6 = stablehlo.dot_general %1, %5